In [1]:
import os
from options.test_options import TestOptions
from data import CreateDataLoader
from models import create_model
from util.visualizer import Visualizer
from util import html
import torch
from models.cycle_gan_model import CycleGANModel
import matplotlib.pyplot as plt
from torchsummary import summary

In [ ]:
opt = TestOptions().parse()
opt.dataroot = 'datasets/horse2zebra/testA'
opt.checkpoints_dir = './checkpoints/'
opt.name = 'horse2zebra_pretrained'
opt.no_dropout = True
opt.model = 'test'
opt.dataset_mode = 'single'
opt.loadSize = 256
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.display_id = -1  # no visdom display
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
model = create_model(opt)

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
dataroot: None
dataset_mode: unaligned
display_id: 1
display_port: 8097
display_server: http://localhost
display_winsize: 256
fineSize: 256
gpu_ids: [0]
how_many: 50
init_type: normal
input_nc: 3
isTrain: False
loadSize: 286
max_dataset_size: inf
model: cycle_gan
nThreads: 4
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
no_dropout: False
no_flip: False
norm: instance
ntest: inf
output_nc: 3
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
suffix: 
verbose: False
which_direction: AtoB
which_epoch: latest
which_model_netD: basic
which_model_netG: resnet_9blocks
-------------- End ----------------
CustomDatasetDataLoader
dataset [SingleImageDataset] was created


In [ ]:
dick = torch.load('/home/toby/Documents/pytorch-CycleGAN-and-pix2pix/checkpoints/yosemite_cyclegan/latest_net_G_A.pth')

for key, value in dick.items():
    print(key)

In [ ]:
from collections import OrderedDict
new_dict = OrderedDict()
for key, value in dick.items():
    names = key.split('.')
    if len(names) == 3 and int(names[1]) < 10:
        new_key = 'module.' + key.replace('model', 'down')
    else:
        new_key = 'module.up'
        new_key += '.' + str(int(names[1]) - 10)
        for name in names[2:]:
            new_key += '.' + name
    new_dict[new_key] = value
    #print(new_key)
#torch.save(new_dict, './checkpoints/horse2zebra_pretrained/updated_net_G.pth')

In [ ]:
for k in model.netG.state_dict().keys():
    print(k)
model.netG.module.load_state_dict(dick)

In [ ]:
data_iter = iter(data_loader)
data = next(data_iter)
result = model.netG(data['A'],cache=True)

In [ ]:
model.netG.module.intermediate.shape
data['A'].shape

In [ ]:
summary(model.netG, (3,256,256))